In [1]:
import os
import PIL.Image
from tensorflow.python.keras.utils.generic_utils import Progbar
from tf_agents.environments import suite_gym, parallel_py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.trajectories.trajectory import Trajectory
from tf_agents.networks import actor_distribution_network
from tf_agents.drivers import dynamic_episode_driver, dynamic_step_driver
from tf_agents.policies import random_tf_policy
import tensorflow as tf
import numpy as np
from reinforcement_learning.sac_training import NumberOfSafetyViolations
from reinforcement_learning import sac_training
from reinforcement_learning import labeling_functions

In [2]:
py_env = suite_gym.load('BipedalWalker-v2')
py_env.render(mode='human')
py_env.reset()


TimeStep(step_type=array(0, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([ 2.7472775e-03, -2.6103160e-05,  2.0305633e-03, -1.5999826e-02,
        9.1790922e-02, -2.6796611e-03,  8.6036706e-01,  3.3826630e-03,
        1.0000000e+00,  3.2197077e-02, -2.6794698e-03,  8.5391629e-01,
        1.8950762e-03,  1.0000000e+00,  4.4081420e-01,  4.4582030e-01,
        4.6142298e-01,  4.8955038e-01,  5.3410304e-01,  6.0246128e-01,
        7.0914918e-01,  8.8593221e-01,  1.0000000e+00,  1.0000000e+00],
      dtype=float32))

In [3]:
bad_state_detection = lambda trajectory: print("bad state!: {}".format(trajectory.observation[..., 0]))\
    if tf.math.abs(trajectory.observation[...,0]) > np.pi / 3 else None

tf_env = tf_py_environment.TFPyEnvironment(py_env)
policy = random_tf_policy.RandomTFPolicy(time_step_spec=tf_env.time_step_spec(), action_spec=tf_env.action_spec())
dynamic_episode_driver.DynamicEpisodeDriver(tf_env, policy, num_episodes=15,
                                            observers=[bad_state_detection, lambda _: py_env.render(mode='human')]).run()

bad state!: [1.0735494]
bad state!: [1.1378732]
bad state!: [1.174754]
bad state!: [1.2165273]
bad state!: [1.2634566]
bad state!: [1.31943]
bad state!: [1.3697844]
bad state!: [1.4257612]
bad state!: [1.4486148]
bad state!: [1.4686921]
bad state!: [1.4952875]
bad state!: [1.5231817]
bad state!: [1.5553536]
bad state!: [1.6080016]
bad state!: [1.6468967]
bad state!: [1.0722704]
bad state!: [1.0935936]
bad state!: [1.1474162]
bad state!: [1.2080934]
bad state!: [1.2711015]
bad state!: [1.281411]
bad state!: [1.2803789]
bad state!: [1.2787379]
bad state!: [1.2594597]
bad state!: [1.2564876]
bad state!: [1.2487124]
bad state!: [1.2456635]
bad state!: [1.2509632]
bad state!: [1.25248]
bad state!: [1.2463536]
bad state!: [1.2301029]
bad state!: [1.2135035]
bad state!: [1.2022425]
bad state!: [1.1893189]
bad state!: [1.1752688]
bad state!: [1.1632671]
bad state!: [1.1410406]
bad state!: [1.1098207]
bad state!: [1.0751128]
bad state!: [1.0602597]
bad state!: [1.055596]
bad state!: [1.0899857]

(TimeStep(step_type=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>, reward=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>, discount=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>, observation=<tf.Tensor: shape=(1, 24), dtype=float32, numpy=
 array([[ 2.7459345e-03,  9.9786403e-06, -1.3011862e-03, -1.6000066e-02,
          9.2471495e-02,  3.0206335e-03,  8.5980916e-01, -9.9893811e-04,
          1.0000000e+00,  3.2791901e-02,  3.0205012e-03,  8.5354382e-01,
         -2.0552513e-03,  1.0000000e+00,  4.4081339e-01,  4.4581950e-01,
          4.6142212e-01,  4.8954949e-01,  5.3410202e-01,  6.0246021e-01,
          7.0914787e-01,  8.8593054e-01,  1.0000000e+00,  1.0000000e+00]],
       dtype=float32)>),
 ())

In [212]:
def bad_state_detection(trajectory):
    # if trajectory.reward[..., 0] <= -100:
    if trajectory.observation[..., 0] < -1. or trajectory.observation[..., 0] > 1:
        py_env.render(mode='human')
        print(trajectory.observation)
        return True
    else:
        return False

walk = True
while walk:
    action = policy.action(time_step=tf_env.current_time_step())
    time_step = tf_env.step(action)
    walk = not bad_state_detection(time_step)


tf.Tensor(
[[-1.139621   -0.15121226  0.19145721 -0.11511189 -0.5889525   0.6279823
  -0.6495559  -1.          0.          0.1954412  -0.14670914  0.18718338
   1.0000001   0.          0.23256062  0.23520169  0.24343319  0.25827238
   0.28177702  0.31784084  0.37412626  0.4673918   0.6417984   1.        ]], shape=(1, 24), dtype=float32)


In [3]:
from reinforcement_learning import labeling_functions

labeling_function = labeling_functions['BipedalWalker-v2']
safety_violations = NumberOfSafetyViolations(labeling_function)
progressbar = Progbar(target=None, interval=0.5, stateful_metrics=['violation'])

tf_env = tf_py_environment.TFPyEnvironment(py_env)
policy = random_tf_policy.RandomTFPolicy(
    time_step_spec=tf_env.time_step_spec(),
    action_spec=tf_env.action_spec())
dynamic_episode_driver.DynamicEpisodeDriver(
    tf_env,
    policy,
    num_episodes=15,
    observers=[safety_violations,
               lambda _: progressbar.add(
                   1, [('violation', safety_violations.average())]),
               lambda _: py_env.render(mode='human')]
).run()

safety_violations._num_episodes

  10233/Unknown - 175s 17ms/step - violation: 7.4667  

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=16.0>

In [2]:
# Parallel environments.
num_parallel_environments = 4
tf_env = tf_py_environment.TFPyEnvironment(
    parallel_py_environment.ParallelPyEnvironment(
    [lambda : suite_gym.load('BipedalWalker-v2')] * num_parallel_environments))
tf_env.reset()

TimeStep(step_type=<tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 0, 0, 0], dtype=int32)>, reward=<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>, discount=<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>, observation=<tf.Tensor: shape=(4, 24), dtype=float32, numpy=
array([[ 2.7473948e-03, -1.6047750e-05,  1.2483769e-03, -1.5999891e-02,
         9.1930076e-02, -1.6474370e-03,  8.6028636e-01,  2.6201985e-03,
         1.0000000e+00,  3.2334823e-02, -1.6473234e-03,  8.5383701e-01,
         1.1650812e-03,  1.0000000e+00,  4.4081411e-01,  4.4582021e-01,
         4.6142289e-01,  4.8955029e-01,  5.3410292e-01,  6.0246116e-01,
         7.0914906e-01,  8.8593203e-01,  1.0000000e+00,  1.0000000e+00],
       [ 2.7466051e-03,  5.8561832e-06, -7.6360616e-04, -1.6000040e-02,
         9.2338696e-02,  1.7726605e-03,  8.5995430e-01, -6.4203050e-06,
         1.0000000e+00,  3.2695595e-02,  1.7725978e-03,  8.5360986e-01,
     

In [3]:
labeling_function = labeling_functions['BipedalWalker-v2']

policy = random_tf_policy.RandomTFPolicy(
    time_step_spec=tf_env.time_step_spec(),
    action_spec=tf_env.action_spec())

safety_violations = NumberOfSafetyViolations(labeling_function)

dynamic_step_driver.DynamicStepDriver(
    tf_env,
    policy,
    num_steps=5000,
    observers=[safety_violations]
).run()

print('Safety violations')
print('episodes', safety_violations._num_episodes)
print('result=', safety_violations.result())
print('average=', safety_violations.average())

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))
Safety violations
episodes <tf.Variable 'num_episodes:0' shape=() dtype=float32, numpy=2.0>
result= <tf.Variable 'num_violations:0' shape=() dtype=float32, numpy=14.0>
average= tf.Tensor(7.0, shape=(), dtype=float32)


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


In [1]:
import importlib
from tf_agents.environments import suite_gym
from reinforcement_learning import labeling_functions
from reinforcement_learning import sac_training

importlib.reload(sac_training)

learner = sac_training.SACLearner(
    env_name='BipedalWalker-v2',
    env_suite=suite_gym,
    num_iterations=int(1e6),
    num_parallel_environments=8,
    labeling_function=labeling_functions['BipedalWalker-v2']
)

Instructions for updating:
`AffineScalar` bijector is deprecated; please use `tfb.Shift(loc)(tfb.Scale(...))` instead.


Instructions for updating:
`AffineScalar` bijector is deprecated; please use `tfb.Shift(loc)(tfb.Scale(...))` instead.


In [ ]:
learner.train_and_eval()

Checkpoint loaded! global_step=2500
Initialize replay buffer...
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))
Start training...
   2495/1000000 [..............................] - ETA: 5:09:22 - loss: 6817.6499 - replay_buffer_frames: 187576.0000   

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./saves/BipedalWalker-v2/stochastic_policy/assets
INFO:tensorflow:Assets written to: ./saves/BipedalWalker-v2/stochastic_policy/assets
INFO:tensorflow:Assets written to: ./saves/BipedalWalker-v2/stochastic_policy/assets
Exception in thread eval:
Traceback (most recent call last):
  File "/home/florentdelgrange/anaconda3/envs/vae-mdp/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/florentdelgrange/anaconda3/envs/vae-mdp/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
TypeError: eval() argument after * must be an iterable, not numpy.int64
INFO:tensor

In [6]:
# Before running this cell, load the single py environment
tf_env = tf_py_environment.TFPyEnvironment(py_env)
stochastic_policy_dir = "../saves/BipedalWalker-v2/policy"
policy = tf.compat.v2.saved_model.load(stochastic_policy_dir)
dynamic_episode_driver.DynamicEpisodeDriver(
    tf_env,
    policy,
    num_episodes=15,
    observers=[lambda _: py_env.render(mode='human')]
).run()

(TimeStep(step_type=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>, reward=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>, discount=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>, observation=<tf.Tensor: shape=(1, 24), dtype=float32, numpy=
 array([[ 2.7460146e-03,  9.4402340e-06, -1.2309741e-03, -1.6000064e-02,
          9.2454262e-02,  2.8576427e-03,  8.5982788e-01, -8.6930749e-04,
          1.0000000e+00,  3.2780085e-02,  2.8575184e-03,  8.5355175e-01,
         -1.9443644e-03,  1.0000000e+00,  4.4081339e-01,  4.4581950e-01,
          4.6142212e-01,  4.8954949e-01,  5.3410202e-01,  6.0246021e-01,
          7.0914787e-01,  8.8593054e-01,  1.0000000e+00,  1.0000000e+00]],
       dtype=float32)>),
 ())

In [3]:
from reinforcement_learning import sac_training

learner = sac_training.SACLearner(
    env_name='BipedalWalker-v2',
    env_suite=suite_gym,
    labeling_function=labeling_functions['BipedalWalker-v2'],
    save_directory_location='..'
)

Instructions for updating:
`AffineScalar` bijector is deprecated; please use `tfb.Shift(loc)(tfb.Scale(...))` instead.


Instructions for updating:
`AffineScalar` bijector is deprecated; please use `tfb.Shift(loc)(tfb.Scale(...))` instead.


In [4]:
variance_multiplier = 5.8
learner.save_permissive_variance_policy(variance_multiplier=variance_multiplier)


Two checkpoint references resolved to different objects (<tf.Variable 'ActorDistributionNetwork/NormalProjectionNetwork/means_projection_layer/kernel:0' shape=(256, 4) dtype=float32, numpy=
array([[ 2.33012915e-01,  2.16516815e-02, -2.54027843e+00,
         1.40148890e+00],
       [ 1.96908563e-01, -2.58657932e-01, -6.78948984e-02,
        -7.04696542e-03],
       [ 1.43984929e-02, -1.50254413e-01,  5.46988882e-02,
        -2.19506975e-02],
       ...,
       [-3.92013550e-01,  1.89930573e-01, -6.88186847e-03,
        -1.54049158e-01],
       [ 7.66966119e-02,  1.27982408e-01,  1.26986355e-01,
        -1.29648492e-01],
       [ 3.65169859e-03,  1.45255895e-02,  3.00898105e-02,
         1.76665757e-03]], dtype=float32)> and <tf.Variable 'ActorDistributionNetwork/NormalProjectionNetwork/means_projection_layer/kernel:0' shape=(256, 4) dtype=float32, numpy=
array([[ 2.33012915e-01,  2.16516815e-02, -2.54027843e+00,
         1.40148890e+00],
       [ 1.96908563e-01, -2.58657932e-01, -6.789


Two checkpoint references resolved to different objects (<tf.Variable 'ActorDistributionNetwork/NormalProjectionNetwork/means_projection_layer/kernel:0' shape=(256, 4) dtype=float32, numpy=
array([[ 2.33012915e-01,  2.16516815e-02, -2.54027843e+00,
         1.40148890e+00],
       [ 1.96908563e-01, -2.58657932e-01, -6.78948984e-02,
        -7.04696542e-03],
       [ 1.43984929e-02, -1.50254413e-01,  5.46988882e-02,
        -2.19506975e-02],
       ...,
       [-3.92013550e-01,  1.89930573e-01, -6.88186847e-03,
        -1.54049158e-01],
       [ 7.66966119e-02,  1.27982408e-01,  1.26986355e-01,
        -1.29648492e-01],
       [ 3.65169859e-03,  1.45255895e-02,  3.00898105e-02,
         1.76665757e-03]], dtype=float32)> and <tf.Variable 'ActorDistributionNetwork/NormalProjectionNetwork/means_projection_layer/kernel:0' shape=(256, 4) dtype=float32, numpy=
array([[ 2.33012915e-01,  2.16516815e-02, -2.54027843e+00,
         1.40148890e+00],
       [ 1.96908563e-01, -2.58657932e-01, -6.789

In [5]:
# Before running this cell, load the single py environment
tf_env = tf_py_environment.TFPyEnvironment(py_env)
stochastic_policy_dir = os.path.join(
    learner.save_directory_location,
    'policy',
    "permissive_variance_policy-multiplier={}".format(
        variance_multiplier)
)
policy = tf.compat.v2.saved_model.load(stochastic_policy_dir)
safety_violations = NumberOfSafetyViolations(
    labeling_function=labeling_functions['BipedalWalker-v2'])

dynamic_episode_driver.DynamicEpisodeDriver(
    tf_env,
    policy,
    num_episodes=2,
    observers=[
        lambda _: py_env.render(mode='human'),
        safety_violations
    ]
).run()

print("avg number of safety violations per episode", safety_violations.average())

avg number of safety violations per episode tf.Tensor(7.6666665, shape=(), dtype=float32)
